### 0. Импорт библиотек
### 1. Модуль прогноза ДВК
### 2. Модуль краткосрочного прогноза
### 3. Вывод результатов

In [1]:
%pylab inline
import numpy as np
import pandas as pd
import pylab as plt
from datetime import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import TensorBoard


from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

import seaborn as sns
sns.set(style="darkgrid")
from IPython.display import Markdown, display
from sklearn.linear_model import Ridge

import ppscore as pps

def printmd(string):
    display(Markdown(string))

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999


RND_SEED=1987

Populating the interactive namespace from numpy and matplotlib


In [ ]:
#function for  choosing a model
def model_selection(feat, obj):
    
    #tree
    tree = DecisionTreeRegressor(random_state=RND_SEED)
    mse = -cross_val_score(tree, feat, obj, cv=cv, 
                       scoring='neg_mean_squared_error').mean()
    best_model = tree
    global min_mse
    min_mse = mse
            
    #fit linear regression for eur
#     linreg = LinearRegression()
#     mse = -cross_val_score(linreg, feat, obj, cv=cv, 
#                        scoring='neg_mean_squared_error').mean()
    #choosing best model
    if mse < min_mse:
        min_mse = mse
        best_model = linreg
        
    #Random Forest regression    
    rfreg = RandomForestRegressor(n_estimators=300)
    # rfreg.fit(X_train.reshape(-1,1), y_train)
    # y_pred = rfreg.predict(X_test.reshape(-1,1))

    # print('R2 score:', r2_score (y_test,y_pred))
    # print('MSE:', mean_squared_error(y_test,y_pred))
    mse = -cross_val_score(rfreg, feat, obj, cv=cv, 
                           scoring='neg_mean_squared_error').mean()
    if mse < min_mse:
        min_mse = mse
        best_model = rfreg
        
    gbreg = GradientBoostingRegressor(max_depth=1, learning_rate=0.1, warm_start=True, n_estimators=250)
    # gbreg.fit(X_train.reshape(-1, 1), y_train)
    # y_pred = gbreg.predict(X_test.reshape(-1, 1))
    mse = -cross_val_score(gbreg, feat, obj, cv=cv, 
                           scoring='neg_mean_squared_error').mean()
    if mse < min_mse:
        min_mse = mse
        best_model = gbreg
    # print('R2 score:', r2_score (y_test,y_pred))
    # print('MSE:', mean_squared_error(y_test,y_pred))
    
    #fit linear regression for eur
    rdg = Ridge(alpha=2)
    # linreg.fit(X_train.reshape(-1,1),y_train)
    # y_pred = linreg.predict(X_test.reshape(-1,1))

    mse = -cross_val_score(rdg, feat, obj, cv=cv, 
                           scoring='neg_mean_squared_error').mean()
    #choosing best model
    if mse < min_mse:
        min_mse = mse
        best_model = rdg
    
    bagreg = BaggingRegressor(n_estimators=30)
    mse = -cross_val_score(bagreg, feat, obj, cv=cv, 
                           scoring='neg_mean_squared_error').mean()

    if mse < min_mse:
        min_mse = mse
        best_model = bagreg
    
    
    
    
 

    return best_model
    

In [ ]:
def baseline_model():
    model = Sequential()
    model.add(Dense(3, input_dim = 1))
    model.add(Activation('relu'))
    model.add(Dense(1, input_dim = 3))
    model.add(Activation('linear'))
    model.compile(loss = 'mean_squared_error', optimizer = 'sgd')
    return model

# 1. Модуль ДВК

In [ ]:
#read file for DVK forecast

df = pd.read_excel('DVK.xlsx', sheet_name='sheet0', header=None)
print(df.shape)
df[0:10]

In [ ]:
#cleaning dataframe
df.drop([0,1,2,3,4,5], axis = 0, inplace = True)
df.head()

In [ ]:
df.reset_index(inplace = True, drop = True)
df.head()

In [ ]:
#creating names for cols
df.columns = ['date','eurusd','eurchf','gbpusd']

In [ ]:
df.head()

In [ ]:
# calculate monthly averages
count = 0
index = 0
sumeur = 0
sumchf = 0
sumgbp = 0
eurusd_mean = []
chf_mean = []
gbp_mean = []

for i in range(df.shape[0]-1):
    if df['date'].iloc[index].month == df['date'].iloc[index+1].month:
        sumeur = sumeur + df['eurusd'].iloc[index]
        sumchf = sumchf + df['eurchf'].iloc[index]
        sumgbp = sumgbp + df['gbpusd'].iloc[index]
        
        count = count + 1
        index = index + 1
    else:
        sumeur = sumeur + df['eurusd'].iloc[index]
        sumchf = sumchf + df['eurchf'].iloc[index]
        sumgbp = sumgbp + df['gbpusd'].iloc[index]
        
        count = count + 1
                
        mean_eur = sumeur/count
        eurusd_mean.append(mean_eur)
        
        mean_chf = sumchf/count
        chf_mean.append(mean_chf)
        
        mean_gbp = sumgbp/count
        gbp_mean.append(mean_gbp)
        
        
        count = 0
        sumchf = 0
        sumgbp = 0
        sumeur = 0
        index = index + 1

        


    
    


In [ ]:
#forming dataframe from monthly means

means = pd.DataFrame(
    {'eurusd':eurusd_mean,
     'eurchf':chf_mean,
     'gbpusd':gbp_mean
    })

In [ ]:
# попробуем потестировать pps. 

means_without_na = means[0:255]
means_without_na
pps.score(means_without_na, "gbpusd", "eurchf")
pps.matrix(means_without_na)

In [ ]:
# создаем массив лагов для каждой валютной пары
X_eurusd = means_without_na['eurusd'][1:]
X_eurchf = means_without_na['eurchf'][1:]
X_gbpusd = means_without_na['gbpusd'][1:]

#создаем массив целевых переменных
y_eurusd = means_without_na['eurusd'][0:254]
y_eurchf = means_without_na['eurchf'][0:254]
y_gbpusd = means_without_na['gbpusd'][0:254]

#Сделаем разбивку на трейн и тест
breakpoint = 30
X_eurusd_train = X_eurusd[breakpoint:]
X_eurusd_test = X_eurusd[0:breakpoint]
X_eurchf_train = X_eurchf[breakpoint:]
X_eurchf_test = X_eurchf[0:breakpoint]
X_gbpusd_train = X_gbpusd[breakpoint:]
X_gbpusd_test = X_gbpusd[0:breakpoint]

y_eurusd_train = y_eurusd[breakpoint:]
y_eurusd_test = y_eurusd[0:breakpoint]
y_eurchf_train = y_eurchf[breakpoint:]
y_eurchf_test = y_eurchf[0:breakpoint]
y_gbpusd_train = y_gbpusd[breakpoint:]
y_gbpusd_test = y_gbpusd[0:breakpoint]

#массив предиктов
predictions = pd.DataFrame(columns = ['fed', 'ecb', 'eurusd', 'eurchf', 'gbpusd', 'bunds2', 'bunds5', 'bunds10', 'ust2', 'ust5', 'ust10', 'eu_libor3', 'us_libor3'], index=['m+0', 'm+1', 'm+2', 'm+3', 'm+4', 'm+5', 'm+6', 'm+7', 'm+8', 'm+9', 'm+10', 'm+11', 'm+12'])


In [ ]:
#функция для отбора нейросетей
def ann_model(Xtrain, ytrain, Xtest, ytest):
    min_mse = 1  
    for n_neurons in range(1,4):
        for i in [1e-2,1e-3,1e-4]:
            model = Sequential()
            model.add(tf.keras.Input(shape=(1,)))
            model.add(Dense(n_neurons, activation='linear'))
            model.add(Dense(1, activation='linear'))
            opt = optimizers.Adam(lr = i)
            model.compile(loss = 'mean_squared_error', optimizer = opt)
            history = model.fit(Xtrain, ytrain, validation_data = (Xtest, ytest), epochs = 50, verbose = 0)

            # Plot training & validation loss values

            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            title_text = 'Feature_name: ' + str(Xtrain[2:8]) + '\n' + 'Model parameters:' + str(n_neurons) + ' neurons, ' + str(i) + ' learning rate'
            plt.title(title_text)
            plt.ylabel('Loss')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Test'], loc='upper left')
            plt.show()
            print('Ошибка на тестовом сете: ', history.history['val_loss'][-1])

            current_mse = history.history['val_loss'][-1]
            if current_mse < min_mse:
                min_mse = current_mse
                best_model = model

    return best_model, min_mse

In [ ]:
#составление массива с прогнозами
def fill_predictions(predictions, currency_pair, model):
    #заполняем раздел в файле прогнозов
    #pred = np.reshape(model.predict([means_without_na[currency_pair][1]]), -1)
    predictions[currency_pair][0] = means_without_na[currency_pair][0]

    for i in range(1, 13):
        pred = np.reshape(ann.predict([float(predictions[currency_pair][i-1])]), -1)
        predictions[currency_pair][i] = pred[0]

    return predictions

In [ ]:
# прогоняем модель на всех трех валютных парах
best_model, min_mse = ann_model(X_eurusd_train, y_eurusd_train,X_eurusd_test, y_eurusd_test)
predictions = fill_predictions(predictions, 'eurusd', best_model)

best_model, min_mse = ann_model(X_gbpusd_train, y_gbpusd_train,X_gbpusd_test, y_gbpusd_test)
predictions = fill_predictions(predictions, 'gbpusd', best_model)

best_model, min_mse = ann_model(X_eurchf_train, y_eurchf_train,X_eurchf_test, y_eurchf_test)
predictions = fill_predictions(predictions, 'eurchf', best_model)

In [ ]:
predictions_vertical = predictions.T
predictions_vertical

In [ ]:
#оценили модель
ann = baseline_model()
history = ann.fit(X_eurchf_train, y_eurchf_train, validation_data = (X_eurchf_test, y_eurchf_test), epochs = 500, verbose = 0)

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
print('Ошибка на тестовом сете: ', history.history['val_loss'][-1])

#заполняем раздел в файле прогнозов
pred = np.reshape(ann.predict([means_without_na['eurchf'][1]]), -1)
predictions['eurchf'][0] = pred[0]

for i in range(1, 7):
    pred = np.reshape(ann.predict([float(predictions['eurchf'][i-1])]), -1)
    predictions['eurchf'][i] = pred[0]
    
predictions

In [ ]:
means.isna().sum()

In [ ]:
cv = KFold(3, random_state=RND_SEED)

In [ ]:
#splitting into train and test

X = means['eurusd'][1:(len(means['eurusd'])-means['eurusd'].isna().sum())-1].values

y = means['eurusd'][0:(len(means['eurusd'])-means['eurusd'].isna().sum())-2].values

X_train = means['eurusd'][int(0.3*(len(means['eurusd']))):(len(means['eurusd'])-means['eurusd'].isna().sum())-1].values
X_test = means['eurusd'][1:int(0.3*(len(means['eurusd'])))].values
y_train = means['eurusd'][int(0.3*(len(means['eurusd']))-1):(len(means['eurusd'])-means['eurusd'].isna().sum())-2].values
y_test = means['eurusd'][0:int(0.3*(len(means['eurusd'])) - 1)].values

In [ ]:
#reshape array
X = X.reshape(-1,1)

In [ ]:
shape(X_train)

In [ ]:
def baseline_model():
    model = Sequential()
    model.add(Dense(5, input_dim = 1))
    model.add(Activation('relu'))
    model.add(Dense(1, input_dim = 5))
    model.add(Activation('exponential'))
    sgd = optimizers.SGD(lr=0.025)
    model.compile(loss = 'mean_squared_error', optimizer = sgd)
    
    return model


In [ ]:


ann = baseline_model()
print(ann.summary())
history = ann.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 150, verbose = 0)

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
print('Ошибка на тестовом сете: ', history.history['val_loss'][-1])

In [ ]:
#forming an array of predictions for 4 months

model = model_selection(X_train.reshape(-1,1),y_train)
model.fit(X_train.reshape(-1,1),y_train)
eurfc = []
eurfc.append(means['eurusd'][0])
for i in range(1,4):
    eurfc.append(model.predict(eurfc[i-1].reshape(-1,1)))
print('прогнозы:',eurfc)
print('MSE:',min_mse)
y_pred = model.predict(X_test.reshape(-1,1))
print(model)

In [ ]:
y_ann = ann.predict(X_test.reshape(-1,1))
ann_mse = mean_squared_error(y_test, y_ann)
y_pred = model.predict(X_test.reshape(-1,1))
model_mse = mean_squared_error(y_test, y_pred)
print('ANN:', ann_mse)
print('Model:', model_mse )
print(shape(y_ann))
print(shape(y_pred))

In [ ]:
# g1 = sns.relplot(kind="line", data=y_test)
# g2 = sns.relplot(kind="line", data=y_pred);

plt.figure(figsize=(15,10))
plt.plot(y_pred, label = 'model selection prediction')
plt.plot(y_test, label = 'fact')
plt.legend(loc="upper left")
plt.grid(True)
plt.gca().invert_xaxis()


In [ ]:
plt.figure(figsize=(15,10))
plt.plot(y_ann, label = 'ANN')
plt.plot(y_test, label = 'test')
plt.legend(loc="upper left")
plt.grid(True)
plt.gca().invert_xaxis()

In [ ]:
#Try to fit linear regression

linreg = LinearRegression()
mse = -cross_val_score(linreg, X_train.reshape(-1,1), y_train, cv=cv, 
                       scoring='neg_mean_squared_error').mean()
linreg = linreg.fit(X_train.reshape(-1,1),y_train)
linreg_pred = linreg.predict(X_test.reshape(-1,1))
linreg_mse = mean_squared_error(y_test,linreg_pred)
print(linreg_mse)

In [ ]:
#splitting into train and test
X_train = means['eurchf'][int(0.2*(len(means['eurchf']))):(len(means['eurchf'])-means['eurchf'].isna().sum())-1].values
X_test = means['eurchf'][1:int(0.2*(len(means['eurchf'])))].values
y_train = means['eurchf'][int(0.2*(len(means['eurchf']))-1):(len(means['eurchf'])-means['eurchf'].isna().sum())-2].values
y_test = means['eurchf'][0:int(0.2*(len(means['eurchf'])) - 1)].values

X = means['eurchf'][1:(len(means['eurchf'])-means['eurchf'].isna().sum())-1].values

y = means['eurchf'][0:(len(means['eurchf'])-means['eurchf'].isna().sum())-2].values

len(X_train), len(X_test)
#reshape array
X = X.reshape(-1,1)

In [ ]:
#forming array of predictions
model = model_selection(X_train.reshape(-1,1),y_train)
model.fit(X_train.reshape(-1,1),y_train)

chffc = []
chffc.append(means['eurchf'][0])
for i in range(1,4):
    chffc.append(model.predict(chffc[i-1].reshape(-1,1)))
print(chffc)
print('MSE:',min_mse)
print(model)


In [ ]:
#splitting into train and test
X_train = means['gbpusd'][int(0.2*(len(means['gbpusd']))):(len(means['gbpusd'])-means['gbpusd'].isna().sum())-1].values
X_test = means['gbpusd'][1:int(0.2*(len(means['gbpusd'])))].values
y_train = means['gbpusd'][int(0.2*(len(means['gbpusd']))-1):(len(means['gbpusd'])-means['gbpusd'].isna().sum())-2].values
y_test = means['gbpusd'][0:int(0.2*(len(means['gbpusd'])) - 1)].values

X = means['gbpusd'][1:(len(means['gbpusd'])-means['gbpusd'].isna().sum())-1].values

y = means['gbpusd'][0:(len(means['gbpusd'])-means['gbpusd'].isna().sum())-2].values

len(X), len(X_test)

#reshape array
X = X.reshape(-1,1)

In [ ]:
# plt.gca().invert_xaxis()
# plt.plot(y_pred, label = 'pred')
# plt.plot(y_test, label = 'test')
# plt.legend(loc="upper left")
# plt.grid(True)

In [ ]:
#forming an array of predictions for 4 months
model = model_selection(X_train.reshape(-1,1),y_train)
model.fit(X_train.reshape(-1,1),y_train)

gbpfc = []
gbpfc.append(means['gbpusd'][0])
for i in range(1,4):
    gbpfc.append(model.predict(gbpfc[i-1].reshape(-1,1)))
print(gbpfc)
print('MSE:',min_mse)
print(model)

# 2. Модуль краткосрочного прогноза

In [2]:
#read file for shortrun forecast and preprocess it

scf = pd.read_excel('shortrun.xlsx', sheet_name='sheet0', header=None)
print(scf.shape)

scf.drop(scf.index[0:6], axis = 0, inplace = True)
scf.drop([2,4,6,7,8,12,13,16,17,18,19,20,25,26,27], axis = 1, inplace = True)
scf.columns = ['date','bunds2', 'bunds5', 'bunds10', 'ust2', 'ust5', 'ust10', 'eu_libor3', 'us_libor3', 'fed_target', 'fed_effect', 'ecb_key', 'ecb_depo']

scf[0:20]

(7835, 28)


,date,bunds2,bunds5,bunds10,ust2,ust5,ust10,eu_libor3,us_libor3,fed_target,fed_effect,ecb_key,ecb_depo
6,2020-05-07 00:00:00,-0.766,-0.739,-0.533,0.1745,0.3544,0.6837,-0.266,0.474,0.25,0.05,0,-0.5
7,2020-05-06 00:00:00,-0.758,-0.724,-0.509,0.1784,0.3718,0.703,-0.266,0.474,0.25,0.05,0,-0.5
8,2020-05-05 00:00:00,-0.805,-0.778,-0.581,0.1882,0.375,0.6619,-0.297,0.474,0.25,0.05,0,-0.5
9,2020-05-04 00:00:00,-0.764,-0.756,-0.565,0.1822,0.3639,0.6336,-0.286,0.50088,0.25,0.05,0,-0.5
10,2020-05-01 00:00:00,-0.777,-0.77,-0.589,0.19,0.3497,0.6118,-0.273,0.54088,0.25,0.05,0,-0.5
11,2020-04-30 00:00:00,-0.777,-0.77,-0.589,0.1956,0.3624,0.6393,-0.273,0.55613,0.25,0.05,0,-0.5
12,2020-04-29 00:00:00,-0.72,-0.676,-0.497,0.2014,0.3671,0.6269,-0.261,0.68663,0.25,0.04,0,-0.5
13,2020-04-28 00:00:00,-0.702,-0.657,-0.471,0.2112,0.375,0.6129,-0.232,0.76013,0.25,0.04,0,-0.5
14,2020-04-27 00:00:00,-0.68,-0.633,-0.456,0.2223,0.4037,0.6605,-0.223,0.84075,0.25,0.04,0,-0.5
15,2020-04-24 00:00:00,-0.716,-0.666,-0.476,0.2245,0.3734,0.6008,-0.192,0.88713,0.25,0.05,0,-0.5


In [3]:
# calculate monthly averages
count = 0
index = 0

sumbund2 = 0
sumbund5 = 0
sumbund10 = 0

sumust2 = 0
sumust5 = 0
sumust10 = 0

sumeu_libor3 = 0
sumus_libor3 = 0

sumfed_target = 0
sumfed_effect = 0
sumecb_key = 0
sumecb_depo = 0

bund2_mean = []
bund5_mean = []
bund10_mean = []
ust2_mean = []
ust5_mean = []
ust10_mean = []

eulibor_mean = []
ustlibor_mean = []

fedtarget_mean = []
fedeffect_mean = []
ecbkey_mean = []
ecbdepo_mean = []


for i in range(scf.shape[0]-1):
    if scf['date'].iloc[index].month == scf['date'].iloc[index+1].month:
                
        sumbund2 = sumbund2 + scf['bunds2'].iloc[index]
        sumbund5 = sumbund5 + scf['bunds5'].iloc[index]
        sumbund10 = sumbund10 + scf['bunds10'].iloc[index]

        sumust2 = sumust2 + scf['ust2'].iloc[index]
        sumust5 = sumust5 + scf['ust5'].iloc[index]
        sumust10 = sumust10 + scf['ust10'].iloc[index]

        sumeu_libor3 = sumeu_libor3 + scf['eu_libor3'].iloc[index]
        sumus_libor3 = sumus_libor3 + scf['us_libor3'].iloc[index]

        sumfed_target = sumfed_target + scf['fed_target'].iloc[index]
        sumfed_effect = sumfed_effect + scf['fed_effect'].iloc[index]
        sumecb_key = sumecb_key + scf['ecb_key'].iloc[index]
        sumecb_depo = sumecb_depo + scf['ecb_depo'].iloc[index]
        
        count = count + 1
        index = index + 1
    else:
        sumbund2 = sumbund2 + scf['bunds2'].iloc[index]
        sumbund5 = sumbund5 + scf['bunds5'].iloc[index]
        sumbund10 = sumbund10 + scf['bunds10'].iloc[index]

        sumust2 = sumust2 + scf['ust2'].iloc[index]
        sumust5 = sumust5 + scf['ust5'].iloc[index]
        sumust10 = sumust10 + scf['ust10'].iloc[index]

        sumeu_libor3 = sumeu_libor3 + scf['eu_libor3'].iloc[index]
        sumus_libor3 = sumus_libor3 + scf['us_libor3'].iloc[index]

        sumfed_target = sumfed_target + scf['fed_target'].iloc[index]
        sumfed_effect = sumfed_effect + scf['fed_effect'].iloc[index]
        sumecb_key = sumecb_key + scf['ecb_key'].iloc[index]
        sumecb_depo = sumecb_depo + scf['ecb_depo'].iloc[index]
        
        count = count + 1
        
        bund2_mean.append(sumbund2/count)
        bund5_mean.append(sumbund5/count)
        bund10_mean.append(sumbund10/count)
        
        ust2_mean.append(sumust2/count)
        ust5_mean.append(sumust5/count)
        ust10_mean.append(sumust10/count)

        eulibor_mean.append(sumeu_libor3/count)
        ustlibor_mean.append(sumus_libor3/count)

        fedtarget_mean.append(sumfed_target/count)
        fedeffect_mean.append(sumfed_effect/count)
        ecbkey_mean.append(sumecb_key/count)
        ecbdepo_mean.append(sumecb_depo/count)
        
        sumbund2 = 0
        sumbund5 = 0
        sumbund10 = 0

        sumust2 = 0
        sumust5 = 0
        sumust10 = 0

        sumeu_libor3 = 0
        sumus_libor3 = 0

        sumfed_target = 0
        sumfed_effect = 0
        sumecb_key = 0
        sumecb_depo = 0
                
        count = 0
        index = index + 1

        


    
    


In [4]:
#forming dataframe from monthly means

scf_means = pd.DataFrame(
    {'bunds2':bund2_mean, 
     'bunds5':bund5_mean, 
     'bunds10':bund10_mean, 
     'ust2':ust2_mean, 
     'ust5':ust5_mean, 
     'ust10':ust10_mean, 
     'eu_libor3':eulibor_mean, 
     'us_libor3':ustlibor_mean, 
     'fed_target':fedtarget_mean, 
     'fed_effect':fedeffect_mean, 
     'ecb_key':ecbkey_mean, 
     'ecb_depo':ecbdepo_mean
      
    })

In [5]:
scf_means.head()

,bunds2,bunds5,bunds10,ust2,ust5,ust10,eu_libor3,us_libor3,fed_target,fed_effect,ecb_key,ecb_depo
0,-0.774000,-0.753400,-0.555400,0.182660,0.362960,0.658800,-0.277600,0.492752,0.250000,0.050000,0.0,-0.5
1,-0.684818,-0.624364,-0.431955,0.220345,0.386968,0.651182,-0.250955,1.103430,0.250000,0.048636,0.0,-0.5
2,-0.805182,-0.706273,-0.520045,0.453459,0.602809,0.875259,-0.416636,1.102355,0.727273,0.625455,0.0,-0.5
3,-0.667750,-0.637050,-0.435100,1.339990,1.333890,1.508785,-0.408850,1.678934,1.750000,1.583500,0.0,-0.5
4,-0.614435,-0.544348,-0.267913,1.520791,1.569139,1.762948,-0.390783,1.823584,1.750000,1.551739,0.0,-0.5


### Нейросеть

In [6]:
predictions['fed'] = 0.25
predictions.iloc[12, 0] = 0.5
predictions['ecb'] = 0
predictions['bunds2'].iloc[0] = scf_means_filled['bunds2'][0]
predictions


NameError: name 'predictions' is not defined

In [7]:
#функция для отбора нейросетей
def rates_model(Xtrain, ytrain, Xtest, ytest):
    min_mse = 1  
    for n_neurons in range(2,6):
        for i in [1e-2,1e-3,1e-4]:
            model = Sequential()
            model.add(tf.keras.Input(shape=(2,)))
            model.add(Dense(n_neurons, activation='linear'))
            model.add(Dense(1, activation='linear'))
            opt = optimizers.Adam(lr = i)
            model.compile(loss = 'mean_squared_error', optimizer = opt)
            history = model.fit(Xtrain, ytrain, validation_data = (Xtest, ytest), epochs = 50, verbose = 0)

            # Plot training & validation loss values

            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            title_text = 'Model parameters:' + str(n_neurons) + ' neurons, ' + str(i) + ' learning rate'
            plt.title(title_text)
            plt.ylabel('Loss')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Test'], loc='upper left')
            plt.show()
            print('Ошибка на тестовом сете: ', history.history['val_loss'][-1])

            current_mse = history.history['val_loss'][-1]
            if current_mse < min_mse:
                min_mse = current_mse
                best_model = model

    return best_model, min_mse

In [10]:
indicators = ['bunds2', 
     'bunds5', 
     'bunds10', 
     'ust2', 
     'ust5', 
     'ust10', 
     'eu_libor3', 
     'us_libor3']
scf_means_filled =  scf_means[0:250]

In [ ]:
#Прогноз Bunds
for item in ['bunds2', 'bunds5', 'bunds10', 'eu_libor3']:
    X = pd.DataFrame()
    y = pd.DataFrame()
    X['ecb_key'] = scf_means_filled['ecb_key'][0:]
    X[item] = scf_means_filled[item][1:]
    y[item] = scf_means_filled[item][0:]
    X = X[0:241][:]
    y = y[0:241]

    breakpoint = 30
    X_train = X[breakpoint:240]
    X_test = X[0:breakpoint]
    y_train = y[breakpoint:240]
    y_test = y[0:breakpoint]

    best_model, min_mse = rates_model(X_train, y_train, X_test, y_test)

    for i in range(1, 13):
        pred = np.reshape(best_model.predict([[predictions['ecb'][i], predictions[item][i-1]]]), -1)
        predictions[item][i] = pred[0]

In [11]:
scf_means_filled.head()

,bunds2,bunds5,bunds10,ust2,ust5,ust10,eu_libor3,us_libor3,fed_target,fed_effect,ecb_key,ecb_depo
0,-0.774000,-0.753400,-0.555400,0.182660,0.362960,0.658800,-0.277600,0.492752,0.250000,0.050000,0.0,-0.5
1,-0.684818,-0.624364,-0.431955,0.220345,0.386968,0.651182,-0.250955,1.103430,0.250000,0.048636,0.0,-0.5
2,-0.805182,-0.706273,-0.520045,0.453459,0.602809,0.875259,-0.416636,1.102355,0.727273,0.625455,0.0,-0.5
3,-0.667750,-0.637050,-0.435100,1.339990,1.333890,1.508785,-0.408850,1.678934,1.750000,1.583500,0.0,-0.5
4,-0.614435,-0.544348,-0.267913,1.520791,1.569139,1.762948,-0.390783,1.823584,1.750000,1.551739,0.0,-0.5


In [25]:

X = pd.DataFrame()
y = pd.DataFrame()


X['ecb_key'] = scf_means_filled['ecb_key'][0:]
X['bunds2'].iloc[0:5] = scf_means_filled['bunds2'].iloc[1:6]
X.head()

KeyError: 'bunds2'

In [ ]:
item = 'bunds2'
X = pd.DataFrame()
y = pd.DataFrame()
X['ecb_key'] = scf_means_filled['ecb_key'][0:]
X[item][0:240] = scf_means_filled[item][1:241]
y[item][0:240] = scf_means_filled[item][0:240]
X = X[0:240][:]
y = y[0:240]

breakpoint = 30
X_train = X[breakpoint:240]
X_test = X[0:breakpoint]
y_train = y[breakpoint:240]
y_test = y[0:breakpoint]
#best_model, min_mse = rates_model(X_train, y_train, X_test, y_test)

In [ ]:
X

In [ ]:
for i in range(1, 13):
    pred = np.reshape(best_model.predict([[predictions['bunds2'][i-1], predictions['ecb'][i]]]), -1)
    predictions['bunds2'][i] = pred[0]

In [ ]:
predictions

In [ ]:
pred = np.reshape(best_model.predict([[predictions['bunds2'][0], predictions['ecb'][1]]]), -1)

In [ ]:
scf_means.notna().sum()

In [ ]:
scf_means_filled =  scf_means[0:250]

In [ ]:
pps.matrix(scf_means_filled)

# 2.1 Прогноз либоров

In [ ]:
scf_means[0:30]

In [ ]:
#splitting into train/test for libor

#using lagged output values to construct X array of eur libor
X = np.nan
X = scf_means.iloc[:, [10,11]]
X['liborlag']  = scf_means.iloc[1:361, [6]]

#moving lagged values to the previous position to get rid of NAN
for i in range (0,359):
    X.loc[i][2] = X.loc[i+1][2]
    
#no need to reshape X here because it is a dataframe,not an array

# array of output
y = scf_means['eu_libor3']

X = X[0:241][:]
y = y[0:241]
X_train = X[40:240]
X_test = X[0:40]
y_train = y[40:240]
y_test = y[0:40]







In [ ]:
# choosing model to predict EU Libor
model = model_selection(X, y)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Forecast:', y_pred[0])
print('Fact:', y[0])
print('RMSE:',min_mse**0.5)
print(model.coef_)
print(model)

In [ ]:
#using lagged output values to construct X array of usd libor
X = np.nan

X = scf_means.iloc[:, [8,9]]
X['liborlag']  = scf_means.iloc[1:361, [7]]

for i in range (0,359):
    X.loc[i][2] = X.loc[i+1][2]

# array of output
y = scf_means['us_libor3']
X = X[0:241][:]
y = y[0:241]
X_train = X[40:359]
X_test = X[0:40]
y_train = y[40:359]
y_test = y[0:40]

In [ ]:
# predict libor usd using lags
from sklearn.model_selection import TimeSeriesSplit
X = scf_means['us_libor3'].shift(-1)
y = scf_means['us_libor3']
X = X[0:359]
y = y[0:359]
tscv = TimeSeriesSplit(n_splits=5)


In [ ]:
#choosing model to predict USD Libor
model = model_selection(X, y)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


print('Forecast:', y_pred[0])
print('Fact:', y[0])
print('RMSE:',min_mse**0.5)

print(model)

In [ ]:
#протестируем нейросеть
model = Sequential()
model.add(Dense(6, input_dim = 3))
model.add(Activation('relu'))
model.add(Dense(3, input_dim = 6))
model.add(Activation('relu'))
model.add(Dense(1, input_dim = 3))
model.add(Activation('exponential'))
sgd = optimizers.SGD(lr=1e-3)
model.compile(loss = 'mean_squared_error', optimizer = sgd)


ann = model
print(ann.summary())
history = ann.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 500, verbose = 0)

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
print('Ошибка на тестовом сете: ', history.history['val_loss'][-1])

In [ ]:
y_pred = model.predict(X_test)


print('Forecast:', y_pred[0])
print('Fact:', y[0])
print('RMSE:',min_mse**0.5)

print(model)

In [ ]:
X.head()

In [ ]:
model.predict([[0.25, 0.19, 0.57]])

# 2.2 Прогноз Bunds

In [ ]:
scf_means.notna().sum()

In [ ]:
#splitting into train/test for Bund2
# forecasting is based on ecb and fed rates, us and eu libor, bund2 lagged values


X = scf_means.iloc[:, [6,7,8,10,11]]
X['bunds2lag']  = scf_means.iloc[1:, [0]]

#using lagged output values to construct X array of eur libor

for i in range (0,241):
    X.loc[i][5] = X.loc[i+1][5]

# array of output
y = scf_means['bunds2']

X = X[0:241][:]
y = y[0:241]
X_train = X[80:240]
X_test = X[0:80]
y_train = y[80:240]
y_test = y[0:80]

In [ ]:
#splitting into train/test for Bund2
# forecasting is based on ecb and fed rates, us and eu libor, bund2 lagged values


X = scf_means.iloc[:, [6,7,8,10,11]]
X['bunds2lag']  = scf_means.iloc[1:, [0]]

#using lagged output values to construct X array of bunds

for i in range (0,241):
    X.loc[i][5] = X.loc[i+1][5]

# array of output
y = scf_means['bunds2']

X = X[0:241][:]
y = y[0:241]
X_train = X[80:240]
X_test = X[0:80]
y_train = y[80:240]
y_test = y[0:80]

In [ ]:
X.head()

In [ ]:
#choosing model to predict bunds2
model = model_selection(X, y)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Forecast:',y_pred[0])
print('Fact:', y[0])
print('RMSE:',min_mse**0.5)
print(model)

In [ ]:
plt.gca().invert_xaxis()
plt.plot(y_pred, label = 'pred')
plt.plot(y_test, label = 'test')
plt.legend(loc="upper left")
plt.grid(True)

In [ ]:
#протестируем нейросеть
model = Sequential()
model.add(Dense(6, input_dim = 6))
model.add(Activation('relu'))
model.add(Dense(3, input_dim = 6))
model.add(Activation('relu'))
model.add(Dense(1, input_dim = 3))
model.add(Activation('exponential'))
sgd = optimizers.SGD(lr=1e-2)
model.compile(loss = 'mean_squared_error', optimizer = sgd)


ann = model
print(ann.summary())
history = ann.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 1500, verbose = 0)

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
print('Ошибка на тестовом сете: ', history.history['val_loss'][-1])

# 2.3 Прогноз T-bills

In [ ]:
scf_means

In [ ]:
#splitting into train/test for tbills5
# forecasting is based   us  libor, fed rates, lagged values


X = scf_means.iloc[:, [7,8]]
X['ust5']  = scf_means.iloc[1:, [4]]
X
#using lagged output values to construct X array 

for i in range (0,241):
    X.loc[i][2] = X.loc[i+1][2]

# array of output
y = scf_means['ust5']

X = X[0:241][:]
y = y[0:241]
X_train = X[40:240]
X_test = X[0:40]
y_train = y[40:240]
y_test = y[0:40]

In [ ]:
X

In [ ]:
#choosing model to predict tbills5
model = model_selection(X, y)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Forecast:',y_pred[0])
print('Fact:', y[0])
print('RMSE:',min_mse**0.5)
print(model)

In [ ]:
plt.gca().invert_xaxis()
plt.plot(y_pred, label = 'pred')
plt.plot(y_test, label = 'test')
plt.legend(loc="upper left")
plt.grid(True)

In [ ]:
model.coef_

In [ ]:
model.intercept_

# 3. Модуль вывода

In [ ]:
print('Прогнозы для ДВК')
print ('\n')
print('Прогноз среднего курса EURUSD на 4 месяца:', [round(float(eurfc[i]),3) for i in range(0,4)])
print('Прогноз среднего курса EURCHF на 4 месяца:', [round(float(chffc[i]),3) for i in range(0,4)])
print('Прогноз среднего курса GBPUSD на 4 месяца:', [round(float(gbpfc[i]),3) for i in range(0,4)])

In [ ]:
print("Средний курс eurusd за текущий месяц:", eurusd_mean[0])
print("Средний курс eurchf за текущий месяц:", chf_mean[0])
print("Средний курс gbpusd за текущий месяц:", gbp_mean[0])

In [ ]:
print('Последние курсы')
df.head()